In [21]:
%pylab qt

import cPickle as pickle
import os
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA
from glob import glob
from itertools import combinations
import pprint

import helpers.classes as ch
import helpers.features as fh
import helpers.display as dh
import helpers.geometry as gh
import helpers.loading as lh
import helpers.windows
import helpers.to_vtk as vh
reload(helpers.windows)
reload(ch)
reload(fh)
reload(dh)
reload(gh)
reload(lh)
reload(vh)
import algorithms.landmarks as la
import algorithms.comparison as ca
reload(la)
reload(ca)

BASE_PATH = os.getcwd()
print("Current base path: {0}".format(BASE_PATH))
DATA_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_registered_outline_new/'

def append_all_marker_distances(outlines):
    for outline in outlines:
        marker_combinations = combinations(range(7), 2)
        outline['landmarks'] = la.get_using_angles(outline['points'])
        outline['landmark_distances'] = {}
        for i, j in marker_combinations:
            outline['landmark_distances'][(i,j)] = np.linalg.norm(outline['landmarks'][i, :] - outline['landmarks'][j, :])

def do_single_comparison(marker_combination_tuple, outlines, pca):
    classes = np.array(map(lambda o: o['class'], outlines))
    unique_classes = np.unique(classes)
    features = np.array([ [ o['landmark_distances'][t] for t in marker_combination_tuple ] for o in outlines ])
    means = [
        np.mean(features[classes == unique_classes[0], :], axis=0),
        np.mean(features[classes == unique_classes[1], :], axis=0),
    ]
    
    if pca:
        pca = PCA(2)
        features = pca.fit_transform(features)
    
    svc = LinearSVC()
    svc.fit(features, classes)

    recall = ca.get_recall(svc, features, classes)
    margin = ca.get_margin(svc)
    rm = recall*margin
    mean_confidence = ca.get_mean_confidence(svc, features, classes)
    
    return {
        'means': means,
        'combination': marker_combination_tuple,
        'recall': recall,
        'margin': margin,
        'rm': rm,
        'mean_confidence': mean_confidence
    }

def do_comparison_of_all_landmarks(outlines):   
    return do_single_comparison(outlines[0]['landmark_distances'].keys(), outlines, pca=True)

def do_comparison_of_landmark_combinations(outlines):
    marker_distance_combinations = combinations(range(7), 2)
    marker_combinations = combinations(marker_distance_combinations, 3)
    results = map(lambda c: do_single_comparison(c, outlines, pca=False), marker_combinations)
    
    results.sort(key=lambda x: -x['mean_confidence'])
    for result in results:
        pprint.pprint(result)

Populating the interactive namespace from numpy and matplotlib
Current base path: /home/stefan/Dropbox/Masterarbeit


In [3]:
loaded = lh.load_files(DATA_PATH)

In [4]:
append_all_marker_distances(loaded)

In [7]:
outline = loaded[17]
landmarks = outline['landmarks']
window = dh.outline(outline, show_direction=True)
pointActor = vh.get_points_actor(landmarks)
window.ren.AddActor(pointActor)
window.vtkWidget.GetRenderWindow().Render()

In [10]:
print(loaded[10]['landmark_distances'])

{(0, 1): 0.69160613704830953, (1, 2): 0.61533109793124874, (2, 5): 2.122484360582106, (1, 3): 1.2420697163702312, (4, 6): 1.4621324499149759, (1, 5): 1.8107081360838668, (4, 5): 0.69882102114297973, (5, 6): 0.77021434430385427, (1, 4): 1.9888830305517491, (2, 4): 2.1007537092112623, (0, 6): 2.2162389183505238, (2, 6): 2.5187883329437275, (0, 5): 2.2080347999376007, (3, 6): 1.7081788576485122, (0, 4): 2.5391074151601924, (2, 3): 1.2663020975053374, (1, 6): 2.050523325094959, (0, 3): 1.8766925365335259, (3, 4): 0.83746141151549514, (0, 2): 1.1902455636543281, (3, 5): 1.0527069522483665}


In [19]:
do_comparison_of_all_landmarks(loaded)

{'combination': [(0, 1),
  (1, 2),
  (2, 5),
  (1, 3),
  (4, 6),
  (1, 5),
  (4, 5),
  (5, 6),
  (1, 4),
  (2, 4),
  (0, 6),
  (2, 6),
  (0, 5),
  (3, 6),
  (0, 4),
  (2, 3),
  (1, 6),
  (0, 3),
  (3, 4),
  (0, 2),
  (3, 5)],
 'margin': 4.1240418795667706,
 'mean_confidence': 0.0088487081105802813,
 'means': [array([ 1.56215386,  1.55881751,  1.58619754,  1.55947411,  1.57263318,
          1.55177652,  1.5835149 ,  1.56647006,  1.58390743,  1.58108761,
          1.5471232 ,  1.58454686,  1.54175781,  1.57368018,  1.56613284,
          1.55750679,  1.54353196,  1.57051031,  1.57318735,  1.58290004]),
  array([ 1.61273197,  1.57271898,  1.56314591,  1.55029296,  1.56987068,
          1.57195037,  1.54283226,  1.5853098 ,  1.57360143,  1.57666404,
          1.55110243,  1.55001012,  1.56306068,  1.56861619,  1.5482256 ,
          1.5570187 ,  1.585094  ,  1.56758357,  1.5649726 ,  1.57069239])],
 'recall': 0.55,
 'rm': 2.268223033761724}

In [22]:
do_comparison_of_landmark_combinations(loaded)

{'combination': ((0, 6), (1, 2), (2, 3)),
 'margin': 0.95294692266747927,
 'mean_confidence': 0.068639193715049923,
 'means': [array([ 2.25517839,  0.54797285,  1.15400526]),
           array([ 2.22566815,  0.59261486,  1.19230463])],
 'recall': 0.75,
 'rm': 0.71471019200060948}
{'combination': ((0, 1), (1, 2), (2, 3)),
 'margin': 0.97393875603482483,
 'mean_confidence': 0.066097585130480474,
 'means': [array([ 0.75643738,  0.54797285,  1.15400526]),
           array([ 0.72288815,  0.59261486,  1.19230463])],
 'recall': 0.775,
 'rm': 0.75480253592698932}
{'combination': ((0, 6), (1, 2), (2, 4)),
 'margin': 0.96360800383554512,
 'mean_confidence': 0.064166555604046743,
 'means': [array([ 2.25517839,  0.54797285,  2.07812896]),
           array([ 2.22566815,  0.59261486,  2.10989718])],
 'recall': 0.875,
 'rm': 0.843157003356102}
{'combination': ((0, 3), (1, 2), (2, 3)),
 'margin': 0.94098467754268444,
 'mean_confidence': 0.063715898740966698,
 'means': [array([ 1.8497894 ,  0.54797285, 